In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm

In [2]:
base_folder =  Path("/media/filer2/T4/Clara/Data/Luisa/")
dest_base_folder = Path("/home/julienb/Documents/Data/Luisa/")

In [3]:
sessions = [f.parent for f in base_folder.glob("**/metadata.yaml")]
len(sessions)
sessions

[PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240828'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240829'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240830'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240902'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_CueRT/Rat101_20240904'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240729'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240730'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240731'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240801'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240802'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240805'),
 PosixPath('/media/filer2/T4/Clara/Data/Luisa/Rat101/opto_MT/Rat101_20240806'),
 PosixPath('/media/filer2

In [4]:
progress = tqdm(sessions)
copy_errors = []
execute_errors = []
for s in progress:
    progress.set_postfix_str(f"Copying {s.stem}")
    dest_folder = dest_base_folder / s.relative_to(base_folder).with_name(f"Session--{s.name}")
    try:
        papermill.execute_notebook("copy_session.ipynb", "copy_session_tmp.ipynb", parameters=
                                   dict(dest_folder = str(dest_folder), session_folder=str(s)), progress_bar=False)
    except Exception as e:
        e.add_note(f"Error for preparing session {s}")
        copy_errors.append(s)
        print(e)
    else:
        progress.set_postfix_str(f"Executing {s.stem}")
        analysis_folder = dest_folder/ "analysis"
        for script in ["events.ipynb", "task_graph.ipynb"]:
            shutil.copy(script, analysis_folder/script)
        (analysis_folder/"events").mkdir(exist_ok=True, parents=True)
        try:
            papermill.execute_notebook(analysis_folder/"events.ipynb", analysis_folder/"events"/"notebook.ipynb", cwd=analysis_folder, progress_bar=False)
        except Exception as e:
            execute_errors.append(s)
            e.add_note(f"Error for executing session {s}")
            print(e)
dict(copy_errors = copy_errors, execute_errors=execute_errors)



  0%|          | 0/137 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
[NbConvertApp] Converting notebook task_graph.ipynb to html
[NbConvertApp] Writing